# KERAS FUNCTIONAL API

## Sequential Class

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras import layers

In [ ]:
seq_model = Sequential()
seq_model.add(layers.Dense(32, activation='relu', input_shape=(64,)))
seq_model.add(layers.Dense(32, activation='relu'))
seq_model.add(layers.Dense(10, activation='softmax'))

seq_model.summary()

## Model Class

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras import layers
from tensorflow.keras import Input

In [ ]:
input_tensor = Input(shape=(64,))
x = layers.Dense(32, activation='relu')(input_tensor)
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)

model = Model(input_tensor, output_tensor)

model.summary()

## Multi-Input Models

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras import layers
from tensorflow.keras import Input

text_voca_size = 10000
question_voca_size = 10000
answer_voca_size = 500

# Reference text
text_input = Input(shape=(None,), dtype='int32', name='text')
embedded_text = layers.Embedding(text_voca_size, 64)(text_input)
encoded_text = layers.LSTM(32)(embedded_text)

# Question
question_input = Input(shape=(None,), dtype='int32', name='question')
embedded_question = layers.Embedding(question_voca_size, 32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

# Concatenate
concatenated = layers.concatenate([encoded_text, encoded_question], axis=-1)

# Answer
answer = layers.Dense(answer_voca_size, activation='softmax')(concatenated)

model = Model([text_input, question_input], answer)

In [ ]:
model.summary()

Feeding data to a multi-input model

In [ ]:
import numpy as np
from keras.utils import to_categorical

num_samples = 1000
max_length = 100

text = np.random.randint(1, text_voca_size, 
                                         size=(num_samples, max_length))

question = np.random.randint(1, question_voca_size, 
                                                 size=(num_samples, max_length))

answers = np.random.randint(0, answer_voca_size, 
                                                size=num_samples)
answers = to_categorical(answers)

H = model.fit([text, question], answers, epochs=10, batch_size=128)

## Multi-Output models

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras import Model

vocabulary_size = 50000
num_income_groups = 10

posts_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_posts = layers.Embedding(256, vocabulary_size)(posts_input)
x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

age_prediction = layers.Dense(1, name='age')(x)
income_prediction = layers.Dense(num_income_groups,
                                                      activation='softmax', name='income')(x)
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)

model = Model(posts_input, 
                         [age_prediction, income_prediction, gender_prediction])

In [ ]:
model.summary()

## Inception Modules

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import Input

x = Input(shape=(224, 224, 3), name='image')

branch_a = layers.Conv2D(128, 1, activation='relu', strides=2)(x)

branch_b = layers.Conv2D(128, 1, activation='relu')(x)
branch_b = layers.Conv2D(128, 3, activation='relu', strides=2)(branch_b)

branch_c = layers.AveragePooling2D(3, strides=2)(x)
branch_c = layers.Conv2D(128, 3, activation='relu')(branch_c)

branch_d = layers.Conv2D(128, 1, activation='relu')(x)
branch_d = layers.Conv2D(128, 3, activation='relu')(branch_d)
branch_d = layers.Conv2D(128, 3, activation='relu', strides=2)(branch_d)

output = layers.concatenate(
                                  [branch_a, branch_b, branch_c, branch_d], axis=-1)

model = Model(x, output)

In [ ]:
model = Model(x, output)

In [ ]:
model.summary()

## Residual Connection


```python
from tensorflow.keras import layers 

x = ...
y = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)

y = layers.add([y, x])  # 원본 x를 출력 특성에 더함
```

```python
from tensorflow.keras import layers 

x = ...
y = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
y = layers.MaxPooling2D(2, strides=2)(y)

# y와 크기를 맞추기 위해 1x1 합성곱을 사용하여 원본 텐서 x를 다운샘플링
residual = layers.Conv2D(128, 1, strides=2, padding='same')(x)

y = layers.add([y, residual])  # 다운샘플링된 x를 출력 특성에 더함
```